In [ ]:

# Install the required library
!pip install pyzipper

import pandas as pd
from cryptography.fernet import Fernet
from io import StringIO
from google.colab import files
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
import pyzipper
import os
import numpy as np

# File paths
encrypted_file = "/content/encrypted_data.csv"  # Encrypted file path (changed to .csv)
decrypted_file = "/content/decrypted_data.csv"  # Decrypted file path
zip_file = "/content/decrypted_data.zip"        # Zip file path
key_file = "/content/encryption_key.key"        # Encryption key path

# Initialize user credentials
USERNAME = ""
PASSWORD = ""

# Function to set username and password
def set_credentials():
    global USERNAME, PASSWORD
    USERNAME = input("Set your username: ")
    PASSWORD = input("Set your password: ")

# Generate and save encryption key
def generate_key():
    key = Fernet.generate_key()
    with open(key_file, 'wb') as kf:
        kf.write(key)
    return key

# Load encryption key from file
def load_key():
    with open(key_file, 'rb') as kf:
        return kf.read()

# Encrypt the CSV file
def encrypt_csv(input_file):
    key = generate_key()
    fernet = Fernet(key)

    # Read CSV and convert to bytes
    df = pd.read_csv(input_file)
    csv_bytes = df.to_csv(index=False).encode()

    # Encrypt and save as .csv
    encrypted = fernet.encrypt(csv_bytes)
    with open(encrypted_file, 'wb') as f:
        f.write(encrypted)

# Decrypt the CSV file
def decrypt_csv(input_file):
    key = load_key()
    fernet = Fernet(key)

    # Read encrypted data
    with open(input_file, 'rb') as f:
        encrypted_data = f.read()

    # Decrypt and convert to DataFrame
    decrypted_data = fernet.decrypt(encrypted_data)
    df = pd.read_csv(StringIO(decrypted_data.decode()))

    return df

# Check user credentials
def check_credentials():
    user = input("Enter username: ")
    pwd = input("Enter password: ")
    return user == USERNAME and pwd == PASSWORD

# Create a password-protected zip file
def create_zip_with_password(password, files_to_zip):
    with pyzipper.AESZipFile(zip_file, 'w', compression=pyzipper.ZIP_DEFLATED, encryption=pyzipper.WZ_AES) as zipf:
        zipf.setpassword(password.encode())  # Set the password
        for file in files_to_zip:
            zipf.write(file, arcname=os.path.basename(file))  # Add files to the zip

# Send the zip file via email
def send_email(to_email, subject, body):
    sender_email = "pallaviss6876@gmail.com"  # Replace with your email
    sender_password = "bfzy veyf yaxi fsdb"  # Use your app password here

    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = to_email
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))

    # Attach the zip file
    with open(zip_file, 'rb') as attachment_file:
        part = MIMEBase('application', 'octet-stream')
        part.set_payload(attachment_file.read())
        encoders.encode_base64(part)  # Encode the attachment
        part.add_header('Content-Disposition', f'attachment; filename={os.path.basename(zip_file)}')
        msg.attach(part)

    # Connect to the server and send the email
    try:
        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()
            server.login(sender_email, sender_password)
            server.send_message(msg)
            print("Email sent successfully.")
    except Exception as e:
        print(f"Failed to send email: {e}")

# Calculate distance between specified objects
def calculate_distance_between_objects(df, obj1_index, obj2_index):
    # Extracting the coordinates for the specified objects
    x1, y1, z1 = df.loc[obj1_index, ['X', 'Y', 'Z']]
    x2, y2, z2 = df.loc[obj2_index, ['X', 'Y', 'Z']]

    # Calculating the distance between the two specified objects
    distance = np.sqrt((x2 - x1)**2 + (y2 - y1)**2 + (z2 - z1)**2)

    return distance

# Main process
set_credentials()

# Upload the CSV file
uploaded = files.upload()  # User uploads the CSV file
for filename in uploaded.keys():
    print(f"Uploaded file: {filename}")
    encrypt_csv(filename)

# Ask for recipient's email
recipient_email = input("Enter the recipient's email address: ")

# Send the decrypted file via email
if check_credentials():
    decrypted_df = decrypt_csv(encrypted_file)

    # Save decrypted data to file
    decrypted_df.to_csv(decrypted_file, index=False)
    print(f"Decrypted data saved to {decrypted_file}.")

    # Ask user for the object indices
    num_objects = len(decrypted_df)
    print(f'There are {num_objects} objects in the dataset.')

    obj1_index = int(input(f'Enter the index of the first object (0 to {num_objects-1}): '))
    obj2_index = int(input(f'Enter the index of the second object (0 to {num_objects-1}): '))

    # Check if the indices are valid
    if 0 <= obj1_index < num_objects and 0 <= obj2_index < num_objects:
        # Calculate distance between the specified objects
        distance = calculate_distance_between_objects(decrypted_df, obj1_index, obj2_index)

        # Save the distance to a new CSV file
        distance_file = f'distance_object_{obj1_index}_{obj2_index}.csv'
        distance_df = pd.DataFrame({
            'Object Pair': [f'Object {obj1_index} - Object {obj2_index}'],
            'Distance (m)': [distance]
        })
        distance_df.to_csv(distance_file, index=False)
        print(f'Distance saved to {distance_file}.')

        # Create a zip file with password protection, including the encrypted file as .csv
        create_zip_with_password(PASSWORD, [decrypted_file, distance_file, encrypted_file])

        # Send the zip file via email
        send_email(recipient_email, "Decrypted Data and Distance Measurement",
                   "Attached is the zip file with the decrypted data, distance measurement, and the encrypted file.")
    else:
        print("Invalid object indices. Please enter valid indices.")
else:
    print("Invalid username or password. Access denied.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 3.0 MB/s eta 0:00:00
